In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import plotly
import plotly.express as px
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [2]:
def convert(currencies, money, currency):
    # Объявляем, что money - глобальная переменная
    # Производим конвертацию - делим количество денег на счету на соответствующий курс
    money = money / currencies[currency]
    return money

currencies = {'USD': 74, 'EUR': 88, 'GBP': 98 , 'CHF': 82}

money = 100000

convert_money = convert(currencies, money, 'USD')  
print(convert_money)  

1351.3513513513512


In [3]:
def sum_lst(lst):
    # Выводим текущее значение lst
    print(lst)
    # Задаём условие выхода из рекурсии
    if len(lst) == 0:
        return 0
    # Во всех других случаях возвращаем
    # сумму первого элемента списка 
    # и результат суммирования оставшихся
    return lst[0] + sum_lst(lst[1:])


my_lst = [10, 21, 24, 12]
print(sum_lst(my_lst))

[10, 21, 24, 12]
[21, 24, 12]
[24, 12]
[12]
[]
67


In [4]:
np.zeros((5, 4))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [5]:

def apply_discounts(products, stocks):
    answer_dict = {}
    for prod_key, prod_value in products.items():
        for stock_key, stock_value in stocks.items():
            if prod_key == stock_key:
                prod_value = prod_value - (prod_value * (int(stock_value.replace('%', ''))/100))
                answer_dict[prod_key] = round(prod_value, 2)
            else:
                answer_dict[prod_key] = round(prod_value, 2)
    return answer_dict

products = {'Oranges (packaged)': 114.99, 'Candy (Rotfront)': 280.0, 'Boiled sausage': 199.99, 'Juice J7 (orange)': 119.99, 'Trout (Seven Seas)': 399.99}
stocks = {'Boiled sausage': '33%', 'Juice J7 (orange)': '12%', 'Trout (Seven Seas)': '18%'}

print(apply_discounts(products, stocks))

## {'Oranges (packaged)': 114.99, 'Candy (Rotfront)': 280.0, 'Boiled sausage': 133.99, 'Juice J7 (orange)': 105.59, 'Trout (Seven Seas)': 327.99}


{'Oranges (packaged)': 114.99, 'Candy (Rotfront)': 280.0, 'Boiled sausage': 133.99, 'Juice J7 (orange)': 105.59, 'Trout (Seven Seas)': 327.99}


In [6]:
solar_data = pd.read_csv('data/solarpower.zip')
display(solar_data.tail())
display(solar_data[solar_data['cum_power'] == -1])

,date,cum_power
659,2018-10-22,28101.0
660,2018-10-23,28109.0
661,2018-10-24,28115.0
662,2018-10-25,28117.0
663,2018-10-26,28120.0


,date,cum_power
45,2017-02-15,-1.0
647,2018-10-10,-1.0


In [7]:
for index, value in enumerate(solar_data['cum_power']):
    if value == -1.0:
        solar_data['cum_power'].iloc[index] = (solar_data['cum_power'].iloc[index-1] + solar_data['cum_power'].iloc[index+1]) / 2

round(solar_data['cum_power'].mean())

C:\Users\Илья Судариков\AppData\Local\Temp\ipykernel_15304\3443794607.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solar_data['cum_power'].iloc[index] = (solar_data['cum_power'].iloc[index-1] + solar_data['cum_power'].iloc[index+1]) / 2


23848

In [13]:
solar_data['day_power'] = solar_data['cum_power'].copy()

list_for_exercise = list(solar_data['day_power'].iloc[::-1])
new_list = []
    
for index, value in enumerate(list_for_exercise):
    if index != 0:
        value = abs(solar_data['cum_power'].iloc[index] - solar_data['cum_power'].iloc[index-1])
        new_list.append(value)
    else:
        new_list.append(0)

solar_data['day_power'] = new_list

solar_data['year_month'] = pd.to_datetime(solar_data['date'], dayfirst = True)
solar_data['year_month'] = solar_data['year_month'].dt.to_period('M')
data_for_diagram = solar_data.groupby('year_month', as_index=False)['day_power'].mean().round()
data_for_diagram['month'] = data_for_diagram['year_month'].dt.month
data_for_diagram['year'] = data_for_diagram['year_month'].dt.year
data_for_diagram

,year_month,day_power,month,year
0,2017-01,3.0,1,2017
1,2017-02,4.0,2,2017
2,2017-03,11.0,3,2017
3,2017-04,16.0,4,2017
4,2017-05,19.0,5,2017
5,2017-06,22.0,6,2017
6,2017-07,18.0,7,2017
7,2017-08,15.0,8,2017
8,2017-09,12.0,9,2017
9,2017-10,6.0,10,2017


In [17]:
pic_1 = px.line(
    data_for_diagram,
    x='month',
    y = 'day_power',
    title='Средняя ежедневная выработка солнечной энергии на каждый месяц-год',
    color='year'
)

pic_1.update_layout(
    xaxis_title = 'Номер месяца',
    yaxis_title = 'Средняя выработка солнечной энергии'
)

pic_1.show()
pic_1.write_html('files_to_share/pic_1.html')